In [1]:
import torch
import open_clip
import IPython.display as display
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import classification_report
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string
from PIL import Image
from joblib import dump
from joblib import load
from tqdm.notebook import tqdm

In [ ]:
# Инсталляции, загрузки
!pip install open_clip_torch transformers
!pip install catboost scikit-learn nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

In [2]:
# Модель OpenCLIP для описания изображений
def create_model():
    model, _, transform = open_clip.create_model_and_transforms(
        model_name="coca_ViT-L-14",
        pretrained="mscoco_finetuned_laion2B-s13B-b90k",
        device="cuda"
    )
    model = model.to('cuda')
    print(next(model.parameters()).device)
    return model, transform

In [14]:
# Описание конкретного изображения по пути к файлу
def describe(image_path, model, transform):
    try:
        with Image.open(image_path) as img:
            img.verify()
            im = Image.open(image_path).convert("RGB")
            im = transform(im).unsqueeze(0)
            im = im.to('cuda').half()
            with torch.no_grad(), torch.amp.autocast('cuda'):
                generated = model.generate(im)
            res = open_clip.decode(generated[0]).split("<end_of_text>")[0].replace("<start_of_text>", "")  
    except (IOError, SyntaxError) as e:
                        print(f"Ошибка при обработке файла {image_path}: {e}")
    return res    

In [6]:
# Создаем датасет со всеми описаниями картинок и категорией, к которой они относятся по имени каталога
def create_dataset(root_dir):
    data = []
    for category in os.listdir(root_dir):
        print(category)
        category_data = []
        category_path = os.path.join(root_dir, category)
        if os.path.isdir(category_path):
            for image_name in os.listdir(category_path):
                print("-- "+image_name)
                image_path = os.path.join(category_path, image_name)
                if os.path.isfile(image_path):
                    try:                                               
                        # Получаем описание картинки
                        description = describe(image_path)                        
                        # Добавляем данные в список
                        data.append({
                            'image_id': image_name,
                            'description': description,
                            'category': category
                        })
                    except (IOError, SyntaxError) as e:
                        print(f"Ошибка при обработке файла {image_path}: {e}")                                
    
    # Создаем pandas DataFrame
    df = pd.DataFrame(data)
    return df

In [4]:
def preprocess_text(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return ' '.join(tokens)

In [21]:
def catboost_learn():
    # Разделение данных на признаки и метки
    X = data['description']
    y = data['category']
    
    # Разделение данных на обучающую и тестовую выборки
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    # Преобразование текстовых данных в числовые признаки с помощью TF-IDF
    vectorizer = TfidfVectorizer(max_features=5000)
    X_train_tfidf = vectorizer.fit_transform(X_train)
    X_test_tfidf = vectorizer.transform(X_test)
    
    # Создание объектов Pool для CatBoost
    train_pool = Pool(data=X_train_tfidf, label=y_train)
    test_pool = Pool(data=X_test_tfidf, label=y_test)
    # Обучение модели CatBoost с использованием GPU
    cb_model = CatBoostClassifier(
        iterations=2000,
        task_type="GPU",
        devices='0',  # Укажите номер GPU, который вы хотите использовать
        verbose=100,
        loss_function='MultiClass'
    )    
    cb_model.fit(train_pool, eval_set=test_pool)
    
    # Предсказание на тестовой выборке
    y_pred = cb_model.predict(test_pool)

    # Оценка модели
    print(classification_report(y_test, y_pred))

    return model, vectorizer

In [5]:
# Ищет 10 похожих изображений в датасете base_path с моделью классификатора CatBoost cb_model
def get_similar_images(image, cb_model, model, transform, vectorizer, base_path):    
    description = describe(image, model, transform)
    print(description)
    category = cb_model.predict(vectorizer.transform([description]))[0][0] 
    catalog = base_path + category
    data = []
    for filename in os.listdir(catalog):
        file_path = os.path.join(catalog, filename)
        if os.path.isfile(file_path):
            try:
                # Получаем описание картинки
                desc = describe(file_path, model, transform)  
                print(desc)
                # Добавляем данные в список
                data.append({
                    'image_id': filename,
                    'description': desc,                
                })
            except (IOError, SyntaxError) as e:
                print(f"Ошибка при обработке файла {image_path}: {e}")        
    # Датасет для хранения id и описания всех картинок в категории, которую указал классификатор
    cdf = pd.DataFrame(data)
    # Векторизация всех описаний, включая заданное изображение    
    tfidf_matrix = vectorizer.fit_transform(cdf["description"] + [description])
    
    # Вычисление косинусного сходства
    cosine_similarities = cosine_similarity(tfidf_matrix[-1], tfidf_matrix[:-1])

    # Получение индексов, отсортированных по убыванию значений сходства
    sorted_indices = np.argsort(cosine_similarities)[::-1]
    
    # Получение 10 индексов с максимальным сходством
    top_10_indices = sorted_indices[0][:10]

    # Формируем пути к файлам изображений по индексам
    files = []
    for index in top_10_indices:
        filename = cdf["image_id"][index]
        file_path = os.path.join(catalog, filename)
        files.append(file_path)  # change to filename later
    return files
   

In [18]:
# Ищет 10 похожих изображений в датасете уже обработанных изображений на трейне с моделью классификатора CatBoost cb_model
def get_similar_images_from_dataset(image, cb_model, model, transform, vectorizer, df, base_path):        
    description = describe(image, model, transform)
    print(description)
    category = cb_model.predict(vectorizer.transform([description]))[0][0] 
    catalog = base_path + category
    cdf = df[df['category'] == category]
    
    # Векторизация всех описаний, включая заданное изображение    
    tfidf_matrix = vectorizer.transform(cdf["description"].tolist() + [description])
    
    # Вычисление косинусного сходства
    cosine_similarities = cosine_similarity(tfidf_matrix[-1], tfidf_matrix[:-1])

    # Получение индексов, отсортированных по убыванию значений сходства
    sorted_indices = np.argsort(cosine_similarities[0])[::-1]
    
    # Получение 10 индексов с максимальным сходством
    top_10_indices = sorted_indices[:10]

    # Формируем пути к файлам изображений по индексам
    files = []
    for index in top_10_indices:
        filename = cdf.iloc[index]["image_id"]
        file_path = os.path.join(catalog, filename)
        files.append(filename)  # change to filename later
    return files

In [ ]:
# Создание датасета и сохранение в файле
root_dir = './train_data_rkn/dataset/'
dataset = create_dataset(root_dir)
dataset.to_csv("dataset_big.csv", sep=',', index=False)

In [6]:
def start(image, df):
    # Создаем модель OpenCLIP
    model, transform = create_model()
    # Загружаем с диска модель CatBoost
    cb_model = CatBoostClassifier()
    cb_model.load_model('catboost_model.cbm')
    # Загружаем Vectorizer
    vectorizer = load('tfidf_vectorizer.joblib')
    files = get_similar_images_from_dataset(image, cb_model, model, transform, vectorizer, df, './train_data_rkn/dataset/')
    return files

In [31]:
# Пакетная обработка множества файлов из директории
def start_batch(dir):
     # Создаем модель OpenCLIP
    model, transform = create_model()
    # Загружаем с диска модель CatBoost
    cb_model = CatBoostClassifier()
    cb_model.load_model('catboost_model.cbm')
    # Загружаем Vectorizer
    vectorizer = load('tfidf_vectorizer.joblib')
    
    df = pd.read_csv("dataset_big.csv")
    data = []
    for file in tqdm(os.listdir(dir), desc="Processing files", total=len(os.listdir(dir)), leave=True, unit="file"):
        print(file)
        full_path = os.path.join(dir, file)        
        res = get_similar_images_from_dataset(full_path, cb_model, model, transform, vectorizer, df, './train_data_rkn/dataset/')
        file_name = os.path.basename(file)
        data.append([file_name, ",".join(res)])
        
    df = pd.DataFrame(data, columns=["image", "recs"])    
    df.to_csv("submission.csv", sep=",", index=False)

start_batch('./test_data_rkn/dataset/')

cuda:0


Processing files:   0%|          | 0/420 [00:00<?, ?file/s]

6ecd5546-99e4-442a-b828-cf46c3367440.jpg
a large artichoke is being cooked in a pot . 
ca5c6ff6-cb2d-449f-900f-bae002ddda4a.jpg
a pair of tires sitting on top of each other . 
5819d823-26b6-4fc3-9449-fcb532bd3d7c.jpg
a close - up of the strings of a harp . 
598ea03e-ef61-44b6-84a1-518b2b6b8666.jpg
a silver spoon on top of a white table . 
e73c7bb0-f6e8-4b4a-8621-2fa113f35bd0.jpg
a bunch of ripe bananas sitting on top of a table . 
4db7516b-7198-4e04-b91e-fc550f82a534.jpg
a drawing of a bunny holding a carrot . 
7ebcdc8b-f24c-45c5-8033-57171d2f4234.jpg
an alarm clock sitting on top of a wooden table . 
5fb0f500-be94-4ff7-a8ca-782e7405c72e.jpg
a close up of a person holding a red lipstick . 
2d7bded8-4a0f-4002-a6b6-66109fe059d7.jpg
different types of bread are shown in this picture . 
ab09206b-7669-4c53-a95b-149b323f5a14.jpg
a drawing of a seahorse showing the anatomy of the hippocampus . 
2faa0c06-5bae-4dbb-91cc-e21edb8bf39c.jpg
a whisk is shown on a white background . 
bec3252a-8c56-4e

/home/user1/environments/hack/lib/python3.10/site-packages/PIL/Image.py:1054: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


a series of apple logos with different years . 
a6debb00-439e-4ef6-891d-f6ad1ba0d923.jpg
a black hat with a black ribbon around it 's neck . 
62c493d8-2062-4867-8d31-c336d7ccedae.jpg
three plastic bags of different colors on a white surface . 
2c7a95fd-91bb-421b-9d2a-3f49af4f89ec.jpg
a light house with a red roof on top of a hill . 
ae37442d-ae9e-4a03-b693-070aac079b4f.jpg
a lion standing on top of a rock . 
2a4af479-3682-4d34-a8b4-355ff90feb8c.jpg
a close - up of an ant on the side of the road . 
c5752d2e-0924-46cb-845d-7e3858feb522.jpg
a close up of a bunch of zucchini on a wooden table . 
789e3beb-f6d7-4c2b-9c12-0cff3454ccac.jpg
a wok with a wooden handle on a white surface . 
48ac9901-ea84-4b09-a358-6b1ff597460a.jpg
a parking meter on the side of the street . 
a0f908ce-3ae5-4806-9cee-cd54c1fd301c.jpg
a black flashlight with a black strap around it 's neck . 
aee4b5c3-b33e-4cb0-9dcb-9c62ecba0f90.jpg
a close up of a lion laying on the ground near a tree . 
52aefa84-ec25-4bf1-9bd0-eac

image_id                          8f79c3a496fa80e7.jpg
description    a rhino is walking on a pile of logs . 
category                                    Rhinoceros
Name: 191, dtype: object

In [25]:
my_submission = pd.read_csv('submission.csv')
my_submission

,image,recs
0,6ecd5546-99e4-442a-b828-cf46c3367440.jpg,"1b243bdbf939cc52.jpg, 1291d8aa69685f09.jpg, d0..."
1,ca5c6ff6-cb2d-449f-900f-bae002ddda4a.jpg,"1d93ff42b6be45c2.jpg, 76a843ac5b6b97fa.jpg, b1..."
2,5819d823-26b6-4fc3-9449-fcb532bd3d7c.jpg,"504bf70967452244.jpg, 1b2210ed9f30b712.jpg, 04..."
3,598ea03e-ef61-44b6-84a1-518b2b6b8666.jpg,"a281f6305dfa14b5.jpg, 023f9a06229a11f6.jpg, 0a..."
4,e73c7bb0-f6e8-4b4a-8621-2fa113f35bd0.jpg,"00568c421d1592fa.jpg, 4050617ad45ad28c.jpg, ff..."
...,...,...
415,75454c3a-f316-4920-8562-011a5ce0d525.jpg,"11d462f2ba33ab60.jpg, 6d2cd0b668c9436d.jpg, bb..."
416,9eb471e7-854b-40ac-9f71-4f7dfce9294c.jpg,"3440988f9690ced6.jpg, 7fe83ce71eb81059.jpg, 00..."
417,458d10c5-acbc-4f0d-9959-36fe9142af28.jpg,"209607df95970762.jpg, 91d93d8dc50043a2.jpg, d5..."
418,6dfe36a2-14a5-4677-93be-03d869b6ca74.jpg,"3ffb0cf84319434b.jpg, 6be578b240c73295.jpg, 39..."


In [26]:
def wrap_in_quotes(recs):
    return f'"{recs}"'

my_submission['recs'] = my_submission['recs'].apply(wrap_in_quotes)
my_submission.to_csv("submission.csv", sep=",", index=False)

In [28]:
my_submission

,image,recs
0,6ecd5546-99e4-442a-b828-cf46c3367440.jpg,"""1b243bdbf939cc52.jpg, 1291d8aa69685f09.jpg, d..."
1,ca5c6ff6-cb2d-449f-900f-bae002ddda4a.jpg,"""1d93ff42b6be45c2.jpg, 76a843ac5b6b97fa.jpg, b..."
2,5819d823-26b6-4fc3-9449-fcb532bd3d7c.jpg,"""504bf70967452244.jpg, 1b2210ed9f30b712.jpg, 0..."
3,598ea03e-ef61-44b6-84a1-518b2b6b8666.jpg,"""a281f6305dfa14b5.jpg, 023f9a06229a11f6.jpg, 0..."
4,e73c7bb0-f6e8-4b4a-8621-2fa113f35bd0.jpg,"""00568c421d1592fa.jpg, 4050617ad45ad28c.jpg, f..."
...,...,...
415,75454c3a-f316-4920-8562-011a5ce0d525.jpg,"""11d462f2ba33ab60.jpg, 6d2cd0b668c9436d.jpg, b..."
416,9eb471e7-854b-40ac-9f71-4f7dfce9294c.jpg,"""3440988f9690ced6.jpg, 7fe83ce71eb81059.jpg, 0..."
417,458d10c5-acbc-4f0d-9959-36fe9142af28.jpg,"""209607df95970762.jpg, 91d93d8dc50043a2.jpg, d..."
418,6dfe36a2-14a5-4677-93be-03d869b6ca74.jpg,"""3ffb0cf84319434b.jpg, 6be578b240c73295.jpg, 3..."


In [30]:
sample_submission = pd.read_csv('./test_data_rkn/sample_submission.csv')
sample_submission

,image,recs
0,e624b22d-6895-4a8d-91a3-19fd7e0f4c93.jpg,"2e2765c8e521ef70.jpg,2e2765c8e521ef70.jpg,2e27..."
1,15d50801-4a15-416b-8de7-8eb96a83d30e.jpg,"2e2765c8e521ef70.jpg,2e2765c8e521ef70.jpg,2e27..."
2,34081ccf-0081-4ddd-bb45-110828fb44dc.jpg,"2e2765c8e521ef70.jpg,2e2765c8e521ef70.jpg,2e27..."
3,189deadc-4205-4b46-ba5e-68a11a16f669.jpg,"2e2765c8e521ef70.jpg,2e2765c8e521ef70.jpg,2e27..."
4,318dbb01-aba0-4038-a4dd-34c099b0f3ef.jpg,"2e2765c8e521ef70.jpg,2e2765c8e521ef70.jpg,2e27..."
...,...,...
503,085432ae-1417-47d1-a6e6-9e95caf8a7d6.jpg,"2e2765c8e521ef70.jpg,2e2765c8e521ef70.jpg,2e27..."
504,99055da0-3615-42a2-8a83-1274e1d0a076.jpg,"2e2765c8e521ef70.jpg,2e2765c8e521ef70.jpg,2e27..."
505,8ffc16c4-6e9e-4a9f-87a5-19811fe61d1c.jpg,"2e2765c8e521ef70.jpg,2e2765c8e521ef70.jpg,2e27..."
506,c2232a78-6d52-4e1b-9dc4-dd38d457217c.jpg,"2e2765c8e521ef70.jpg,2e2765c8e521ef70.jpg,2e27..."
